In [0]:
-- DLT pipeline: Bronze → Silver → Gold with expectations.
-- This file is run by the pipeline (not by the SQL warehouse directly).

-- BRONZE (choose ONE source):

-- (A) If you created raw.taxi_raw via Auto Loader (01_auto_loader_bronze.sql):
CREATE OR REFRESH STREAMING TABLE raw.taxi_bronze
AS SELECT * FROM main_nyctaxi.raw.taxi_raw;

-- (B) If you want the lowest-friction samples path:
-- CREATE OR REFRESH STREAMING TABLE raw.taxi_bronze
-- (CONSTRAINT trip_distance_positive EXPECT (trip_distance > 0) ON VIOLATION DROP ROW)
-- AS SELECT * FROM STREAM(samples.nyctaxi.trips);

-- SILVER: typing + pruning.
CREATE OR REFRESH STREAMING TABLE ref.trips_clean
AS
SELECT
  CAST(tpep_pickup_datetime  AS TIMESTAMP) AS pickup_ts,
  CAST(tpep_dropoff_datetime AS TIMESTAMP) AS dropoff_ts,
  DATE(CAST(tpep_pickup_datetime AS TIMESTAMP)) AS pickup_date,
  CAST(passenger_count AS INT)  AS passenger_count,
  CAST(trip_distance  AS DOUBLE) AS trip_distance,
  CAST(fare_amount    AS DOUBLE) AS fare_amount,
  vendor_id, pickup_zip, dropoff_zip
FROM LIVE.raw.taxi_bronze
WHERE trip_distance IS NOT NULL AND fare_amount IS NOT NULL;

-- Expectations on the cleaned stream.
CREATE OR REFRESH STREAMING TABLE ref.trips_valid
(
  CONSTRAINT vendor_not_null EXPECT (vendor_id IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT nonnegative_fare EXPECT (fare_amount >= 0)     ON VIOLATION DROP ROW
)
AS SELECT * FROM LIVE.ref.trips_clean;

-- GOLD: BI aggregates as MVs (eligible on Pro/Serverless SQL or inside the pipeline).
CREATE OR REPLACE MATERIALIZED VIEW mart.daily_kpis AS
SELECT
  pickup_date,
  COUNT(*)                           AS trip_count,
  ROUND(AVG(fare_amount), 2)         AS avg_fare,
  ROUND(AVG(trip_distance), 3)       AS avg_trip_miles,
  ROUND(SUM(fare_amount), 2)         AS total_revenue,
  ROUND(SUM(fare_amount) / NULLIF(SUM(trip_distance), 0), 3) AS revenue_per_mile
FROM LIVE.ref.trips_valid
GROUP BY pickup_date;
